# SSZ Schumann Resonance Analysis

**Segmented Spacetime (SSZ) Analysis of Schumann Resonances**

This notebook demonstrates the SSZ analysis pipeline for Schumann resonances.

© 2025 Carmen Wrede & Lino Casu
Licensed under the Anti-Capitalist Software License v1.4

## 1. Setup

In [ ]:
# Clone repository
!git clone https://github.com/error-wtf/ssz-schuhman-experiment.git
%cd ssz-schuhman-experiment

# Install dependencies
!pip install -q -r requirements.txt
!pip install -q -e .

print("Setup complete!")

## 2. Run Unit Tests

In [ ]:
!python -m pytest tests/ -v --tb=short | tail -20

## 3. Generate Synthetic Data with SSZ Signal

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from ssz_schumann.data_io.schumann_sierra_nevada import create_synthetic_schumann_data
from ssz_schumann.models.classical_schumann import f_n_classical, compute_eta0_from_mean_f1
from ssz_schumann.models.ssz_correction import delta_seg_from_observed, check_mode_consistency

# Generate synthetic data with 3% SSZ signal
ds = create_synthetic_schumann_data(
    start="2016-01-01",
    end="2016-06-30",
    freq="1h",
    eta_0=0.74,
    delta_seg_amplitude=0.03,
    noise_level=0.005,
)

# Convert to DataFrame
df = ds.to_dataframe().reset_index()
print(f"Generated {len(df)} data points")
print(f"Columns: {list(df.columns)}")

## 4. SSZ Analysis

In [ ]:
# Calibrate eta_0 from f1
eta_0 = compute_eta0_from_mean_f1(df['f1'].values)
print(f"Calibrated eta_0 = {eta_0:.6f}")

# Compute delta_seg for each mode
delta_seg_dict = {}
for n in [1, 2, 3]:
    f_obs = df[f'f{n}'].values
    f_class = f_n_classical(n, eta_0)
    delta_seg_dict[n] = delta_seg_from_observed(f_obs, f_class)
    print(f"Mode {n}: mean delta_seg = {np.mean(delta_seg_dict[n]):.6f}")

# Check mode consistency (SSZ signature)
consistency = check_mode_consistency(delta_seg_dict)
print(f"\nSSZ Analysis Results:")
print(f"  Mean Correlation: {consistency['mean_correlation']:.4f}")
print(f"  SSZ Score: {consistency['ssz_score']:.4f}")
print(f"  Interpretation: {consistency['interpretation']}")

## 5. Visualization

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Plot 1: Frequencies over time
ax = axes[0, 0]
for n in [1, 2, 3]:
    ax.plot(df['time'], df[f'f{n}'], label=f'f{n}', alpha=0.7)
ax.set_xlabel('Time')
ax.set_ylabel('Frequency (Hz)')
ax.set_title('Schumann Resonance Frequencies')
ax.legend()

# Plot 2: delta_seg for each mode
ax = axes[0, 1]
for n in [1, 2, 3]:
    ax.plot(df['time'], delta_seg_dict[n], label=f'Mode {n}', alpha=0.7)
ax.axhline(0, color='k', linestyle='--', alpha=0.3)
ax.set_xlabel('Time')
ax.set_ylabel('delta_seg')
ax.set_title('SSZ Segmentation Parameter')
ax.legend()

# Plot 3: Mode correlation
ax = axes[1, 0]
ax.scatter(delta_seg_dict[1], delta_seg_dict[2], alpha=0.3, s=5, label='Mode 1 vs 2')
ax.scatter(delta_seg_dict[1], delta_seg_dict[3], alpha=0.3, s=5, label='Mode 1 vs 3')
ax.plot([-0.1, 0.1], [-0.1, 0.1], 'k--', alpha=0.5, label='1:1 line')
ax.set_xlabel('delta_seg (Mode 1)')
ax.set_ylabel('delta_seg (Mode 2, 3)')
ax.set_title('Mode Correlation (SSZ Signature)')
ax.legend()

# Plot 4: True vs recovered delta_seg
ax = axes[1, 1]
if 'delta_seg_true' in df.columns:
    ax.scatter(df['delta_seg_true'], delta_seg_dict[1], alpha=0.3, s=5)
    ax.plot([-0.1, 0.1], [-0.1, 0.1], 'r--', label='Perfect recovery')
    ax.set_xlabel('True delta_seg')
    ax.set_ylabel('Recovered delta_seg (Mode 1)')
    ax.set_title('SSZ Signal Recovery')
    ax.legend()

plt.tight_layout()
plt.show()

## 6. Model Comparison (Classical vs SSZ)

In [ ]:
from ssz_schumann.analysis.model_comparison import compare_models, print_comparison_summary

# Prepare data for model comparison
f_obs = {
    1: df['f1'].values,
    2: df['f2'].values,
    3: df['f3'].values,
}

# Compare models
result = compare_models(f_obs)
print_comparison_summary(result)

## 7. Physical SSZ Model

In [ ]:
from ssz_schumann.models.physical_ssz import (
    print_physical_model_summary,
    predict_ssz_signature,
    delta_seg_from_proxies,
)

# Print physical model summary
print_physical_model_summary()

# Predict SSZ signature for different conditions
print("\nSSZ Predictions for Different Conditions:")
print(f"  Quiet Sun (F10.7=70, Kp=1): delta_seg = {delta_seg_from_proxies(70, 1):.6f}")
print(f"  Active Sun (F10.7=200, Kp=3): delta_seg = {delta_seg_from_proxies(200, 3):.6f}")
print(f"  Storm (F10.7=150, Kp=7): delta_seg = {delta_seg_from_proxies(150, 7):.6f}")

## 8. Run Full Validation

In [ ]:
!python scripts/run_full_validation.py 2>&1 | tail -50

## Summary

This notebook demonstrated:

1. **Synthetic Data Generation** with known SSZ signal
2. **SSZ Analysis** - extracting delta_seg from frequencies
3. **Mode Consistency Check** - verifying SSZ signature
4. **Model Comparison** - Classical vs SSZ using AIC/BIC
5. **Physical Model** - connecting SSZ to ionospheric properties

The key SSZ prediction is that **delta_seg should be mode-independent**, meaning all Schumann modes show the same relative frequency shift.

---
© 2025 Carmen Wrede & Lino Casu